In [ ]:
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
from torch.autograd import Variable
import numpy as np
import pandas as pd

In [ ]:
from torch.utils.data.dataset import Dataset


label_idx = {'Iris-setosa': 0, 'Iris-versicolor': 1, 'Iris-virginica': 2}


class IrisDataset(Dataset):

    def __init__(self, data):
        self.data = data
           
    def __getitem__(self, index):
        item = self.data.iloc[index].values
        return (item[0:4].astype(np.float32), item[4].astype(np.int))

    def __len__(self):
        return self.data.shape[0]


def get_datasets(iris_file, train_ratio=0.80):

    labels = {'class': label_idx}
    data = pd.read_csv(iris_file)
    data.replace(labels, inplace=True)

    train_df = data.sample(frac=train_ratio, random_state=3)
    test_df = data.loc[~data.index.isin(train_df.index), :]

    return IrisDataset(train_df), IrisDataset(test_df)

In [ ]:
!head data/iris.data.txt

### Fully Connected Feed Fwd Net

In [ ]:
class IrisNet(nn.Module):
    
    def __init__(self, input_size, hidden1_size, hidden2_size, num_classes):
        super(IrisNet, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden1_size)
        self.relu1 = nn.ReLU()
        self.fc2 = nn.Linear(hidden1_size, hidden2_size)
        self.relu2 = nn.ReLU()
        self.fc3 = nn.Linear(hidden2_size, num_classes)
    
    def forward(self, x):
        out = self.fc1(x)
        out = self.relu1(out)
        out = self.fc2(out)
        out = self.relu2(out)
        out = self.fc3(out)
        return out

In [ ]:
model = IrisNet(4, 100, 50, 3).cuda()
print(model)

### Creating the data loader

In [ ]:
batch_size = 60
iris_data_file = 'data/iris.data.txt'

In [ ]:
# Get the datasets
train_ds, test_ds = get_datasets(iris_data_file)

print("training set length", len(train_ds))
print("test set length", len(test_ds))

train_loader = torch.utils.data.DataLoader(dataset = train_ds, batch_size = batch_size, shuffle = True)
test_loader = torch.utils.data.DataLoader(dataset = test_ds, batch_size = batch_size, shuffle = True)

In [ ]:
criterion = nn.CrossEntropyLoss()
learning_rate = 0.001

optimizer = torch.optim.SGD(model.parameters(), lr = learning_rate, nesterov=True, momentum=0.9, dampening=0)

## Training Loop

In [ ]:
# 2 loops outer loop executes the epochs. Inner loop executes the iterations per epoch.
num_epochs = 500
train_loss = []
test_loss = []
train_accuracy = []
test_accuracy = []

for epoch in range(num_epochs):
    
    train_correct = 0
    train_total = 0
    
    for i, (items, classes) in enumerate(train_loader):
        # Each batch is a tuple. First element is a float tensor containing all the dependent variables for each batch
        # Second element of tuple
        # Convert torch tensor to variable
        items = Variable(items.cuda())
        classes = Variable(classes.cuda())
        model.train()
        # Clear off gradients from past operations
        optimizer.zero_grad()
        # Do the forward pass
        outputs = model(items)
        # Calculate the loss
        loss = criterion(outputs, classes)
        # Calculate the gradients with the help of back propagation
        loss.backward()
        # Ask the opitmizer to update the parameters on the basis of the gradients
        optimizer.step()
        
        # Record the correct predictions for training data
        train_total += classes.size(0)
        _, predicted = torch.max(outputs.data, 1)
        train_correct += (predicted == classes.data).sum()
        print('Epoch %d/%d, Iteration %d/%d, Loss: %.4f'%(epoch+1, num_epochs, i+1, len(train_ds)//batch_size, loss.data[0]))
    
    model.eval()
    train_loss.append(loss.data[0])
    
    #Record the training accuracy
    train_accuracy.append(100*train_correct/train_total)
    
    #Check on the test set
    test_items = torch.FloatTensor(test_ds.data.values[:,0:4])
    test_classes = torch.LongTensor(test_ds.data.values[:,4])
    outputs = model(Variable(test_items.cuda()))
    loss = criterion(outputs, Variable(test_classes.cuda()))
    test_loss.append(loss.data[0])
    
    #Record the testing accuracy
    _, predicted = torch.max(outputs.data,1)
    total = test_classes.size(0)
    correct = (predicted==test_classes.cuda()).sum()
    test_accuracy.append((100*correct/total))

Loss vs Iterations Plot

In [ ]:
fig = plt.figure(figsize=(12,8))
plt.plot(train_loss, label = 'train_loss')
plt.plot(test_loss, label = 'test_loss')
plt.title("Train and test loss")
plt.legend()
plt.show()

Plotting train and test set accuracy

In [ ]:
fig = plt.figure(figsize=(12,8))
plt.plot(train_accuracy, label = 'train_accuracy')
plt.plot(test_accuracy, label = 'test_accuracy')
plt.title("Train and test accuracy")
plt.legend()
plt.show()

Persist model to disk

In [ ]:
torch.save(model.state_dict(),"./fwd_net.pth")

Load the model

In [ ]:
net = IrisNet(4, 100, 50, 3)
net.load_state_dict(torch.load("./fwd_net.pth"))
net.eval()

In [ ]:
item = [[5.1,3.5,1.4,0.2]]
expected_class = 0 # Iris-Setosa

In [ ]:
output = net(Variable(torch.FloatTensor(item)))

In [ ]:
_, predicted_class = torch.max(output.data, 1)
print(predicted_class.numpy())
print('Predicted class:', predicted_class.numpy()[0])
print('Expected class:', expected_class)